# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import io





In [15]:
url = "https://en.wikipedia.org/wiki/List_of_most-streamed_songs_on_Spotify"
response = requests.get(url, verify=False)
print(response.status_code)
html = io.StringIO(response.text) 
tables = pd.read_html(html)
print(f"{len(tables)} tables were found.")


/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


200
27 tables were found.


In [16]:

df = tables[0]  
df.head()  

,Rank,Song,Artist(s),Streams (billions),Release date,Ref.
0,1,"""Blinding Lights""",The Weeknd,4.845,29 November 2019,[2]
1,2,"""Shape of You""",Ed Sheeran,4.366,6 January 2017,[3]
2,3,"""Someone You Loved""",Lewis Capaldi,3.908,8 November 2018,[4]
3,4,"""Starboy""",The Weeknd and Daft Punk,3.899,21 September 2016,[5]
4,5,"""As It Was""",Harry Styles,3.882,1 April 2022,[6]


In [17]:
df.columns = ["Rank", "Song", "Artist", "Streams (billions)", "Date released", "Reference"]

df["Song"] = df["Song"].str.replace(r"\[.*?\]", "", regex=True)
df["Artist"] = df["Artist"].str.replace(r"\[.*?\]", "", regex=True)

df = df[df["Streams (billions)"].astype(str).str.contains(r"^\d+(?:\.\d+)?$", na=False)].copy()

df["Streams (billions)"] = df["Streams (billions)"].astype(float)

df["Date released"] = pd.to_datetime(df["Date released"], errors="coerce")

df


    

,Rank,Song,Artist,Streams (billions),Date released,Reference
0,1,"""Blinding Lights""",The Weeknd,4.845,2019-11-29,[2]
1,2,"""Shape of You""",Ed Sheeran,4.366,2017-01-06,[3]
2,3,"""Someone You Loved""",Lewis Capaldi,3.908,2018-11-08,[4]
3,4,"""Starboy""",The Weeknd and Daft Punk,3.899,2016-09-21,[5]
4,5,"""As It Was""",Harry Styles,3.882,2022-04-01,[6]
...,...,...,...,...,...,...
95,96,"""Too Good at Goodbyes""",Sam Smith,2.305,2017-09-08,[97]
96,97,"""Stay with Me""",Sam Smith,2.296,2014-04-14,[98]
97,98,"""Someone Like You""",Adele,2.296,2011-01-24,[99]
98,99,"""Havana""",Camila Cabello and Young Thug,2.284,2017-08-03,[100]


In [18]:
conn = sqlite3.connect("spotify_top_songs.db")
df.to_sql("most_streamed", conn, if_exists="replace", index=False)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM most_streamed")
print("Rows inserted:", cursor.fetchone()[0])

conn.commit()
conn.close()

Rows inserted: 100


In [13]:
top_10_songs = df.nlargest(10, 'Song') 
top_10_songs

TypeError: Column 'Song' has dtype object, cannot use method 'nlargest' with this dtype